In [1]:
# load libraries
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks", color_codes=True)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
class DataCleaner():

    def clean_uom_space(self,uom_space_df):
        uom_space_df['Campus Code']=uom_space_df['Campus Code'].astype(str).str.strip()
        uom_space_df['Building Code']=uom_space_df['Building Code'].astype(str).str.strip()
        uom_space_df['Building Name']=uom_space_df['Building Name'].astype(str).str.strip()
        uom_space_df['Room Type']=uom_space_df['Room Type'].astype(str).str.strip()
        uom_space_df['Room Category']=uom_space_df['Room Category'].astype(str).str.strip()
        uom_space_df['Floor Code']=uom_space_df['Floor Code'].astype(str).str.strip()
        uom_space_df['Room Code']=uom_space_df['Room Code'].astype(str).str.strip()
        return uom_space_df

    def clean_rm_category_type(self,rm_category_type_df):
        rm_category_type_df['Room Type']=rm_category_type_df['Room Type'].astype(str).str.strip()
        rm_category_type_df['Room Category']=rm_category_type_df['Room Category'].astype(str).str.strip()
        rm_category_type_df['Room Type Abbreviation']=rm_category_type_df['Room Type Abbreviation'].str.lower().str.strip()
        rm_category_type_df['Description']=rm_category_type_df['Description'].str.lower().str.strip()
        rm_category_type_df['Room Type Definition']=rm_category_type_df['Room Type Definition'].str.lower().str.strip()
        return rm_category_type_df

    def clean_floor_data(self,floor_df):
        floor_df['Building Code'] = floor_df['Building Code'].astype(str).str.strip()
        floor_df['Floor Code'] = floor_df['Floor Code'].astype(str).str.strip()
        floor_df['Floor Name'] = floor_df['Floor Name'].astype(str).str.strip()
        return floor_df

    def clean_em_location(self,em_location_df):
        em_location_df['Floor Code'] = em_location_df['Floor Code'].astype(int)
        em_location_df['Floor Code'] = em_location_df['Floor Code'].astype(str).str.strip()
        return em_location_df

    def clean_av_equipment(self,av_equipment_df):
        av_equipment_df['Room Type'] = av_equipment_df['Room Type'].astype(str).str.strip()
        av_equipment_df['Room Code'] = av_equipment_df['Room Code'].astype(str).str.strip()
        av_equipment_df['Building Code'] = av_equipment_df['Building Code'].astype(str).str.strip()
        av_equipment_df['Campus Code'] = av_equipment_df['Campus Code'].astype(str).str.strip()
        av_equipment_df['Equip. Status'] = av_equipment_df['Equip. Status'].astype(str).str.strip()
        av_equipment_df['Floor Code'] = av_equipment_df['Floor Code'].astype(int).astype(str).str.strip()
        return av_equipment_df

    def clean_timetable_data(self,timetable_df):
        # remove all NaN rows
        timetable_df = timetable_df.dropna(how='all')

        # drop duplicate records
        timetable_df = timetable_df.drop_duplicates()

        # dropping classes whose location is not planned
        timetable_df = timetable_df[timetable_df['Host Key of Allocated Locations'].notna()]

        # dropping classes whose location is online option
        timetable_df = timetable_df[timetable_df['Host Key of Allocated Locations']!='Online option.']

        # dropping classes with off-site location
        timetable_df = timetable_df[timetable_df['Name of Zone of Allocated Locations']!='Off-Site']
        return timetable_df

    def clean_meeting_room_usage(self,meeting_room_usage_df):
        meeting_room_usage_df = meeting_room_usage_df[meeting_room_usage_df['Campus Code'].notna()]
        meeting_room_usage_df = meeting_room_usage_df[meeting_room_usage_df['Building Code'].notna()]
        meeting_room_usage_df = meeting_room_usage_df[meeting_room_usage_df['Floor Code'].notna()]
        meeting_room_usage_df = meeting_room_usage_df[meeting_room_usage_df['Room Code'].notna()]

        meeting_room_usage_df['Campus Code'] = meeting_room_usage_df['Campus Code'].astype(str).str.strip()
        meeting_room_usage_df['Building Code'] = meeting_room_usage_df['Building Code'].astype(str).str.strip()
        meeting_room_usage_df['Building Name'] = meeting_room_usage_df['Building Name'].astype(str).str.strip()
        meeting_room_usage_df['Floor Code'] = meeting_room_usage_df['Floor Code'].astype(int).astype(str).str.strip()
        meeting_room_usage_df['Room Code'] = meeting_room_usage_df['Room Code'].astype(str).str.strip()
        return meeting_room_usage_df

In [4]:
class DataMutator():

    def mutate_em_location(self,em_location_df):
        # mutate room codes
        for idx,row in em_location_df.iterrows():
            if "." in row['Room Code']:
                code = row['Room Code'].split(".")[0]
                em_location_df.at[idx,'Room Code'] = code
        return em_location_df

    def mutate_timetable_data(self,timetable_df):
        building_codes = []
        room_codes = []
        campus_codes = []
        class_duration_minutes = []
        for idx,row in timetable_df.iterrows():
            s = row['Host Key of Allocated Locations'].split('-')
            building_codes.append(s[0])
            room_codes.append(s[1])
            c = row['Name of Allocated Locations'].split('-')[0]
            if c == 'zzzPAR':
                c = 'PAR'
            campus_codes.append(c)
            d = row['Duration as duration']
            class_duration_minutes.append(d.hour * 60 + d.minute)
        timetable_df['Building Code'] = building_codes
        timetable_df['Room Code'] = room_codes
        timetable_df['Campus Code'] = campus_codes
        timetable_df['Class Duration In Minutes'] = class_duration_minutes
        return timetable_df

In [5]:
class DataMerger():

    def get_merged_space_data(self,uom_space_df,rm_category_type_df, floor_df):
        uom_space_df_enhanced = pd.merge(uom_space_df,floor_df,on=['Building Code','Floor Code'])
        print("# Merge - uom_space + floor_data")
        print((uom_space_df.shape, uom_space_df_enhanced.shape))
        print('Unable to merge records:',uom_space_df.shape[0]-uom_space_df_enhanced.shape[0])
        merged_space_data_df = pd.merge(uom_space_df_enhanced,rm_category_type_df,on=['Room Category','Room Type'])
        print("# Merge - enhanced_uom_space + rm_category_type")
        print((uom_space_df_enhanced.shape, merged_space_data_df.shape))
        print('Unable to merge records:',uom_space_df_enhanced.shape[0]-merged_space_data_df.shape[0])
        return merged_space_data_df

    def get_merged_em_location_data(self,em_location_df,merged_space_data_df):
        print("# merge - space_data + em_location")
        merged_em_location_df = pd.merge(em_location_df,merged_space_data_df,on=['Building Code','Floor Code','Room Code'])
        print((em_location_df.shape, merged_em_location_df.shape))
        print('Unable to merge records:',em_location_df.shape[0]-merged_em_location_df.shape[0])
        return merged_em_location_df

    def get_merged_av_equipment_data(self,av_equipment_df,merged_space_data_df):
        print("# merge - space_data + av_equipment")
        merged_av_equipment_df = pd.merge(av_equipment_df,merged_space_data_df,on=['Campus Code','Building Code','Floor Code','Room Code'])
        print((av_equipment_df.shape, merged_av_equipment_df.shape))
        print('Unable to merge records:',av_equipment_df.shape[0]-merged_av_equipment_df.shape[0])
        return merged_av_equipment_df

    def get_merged_timetable_data(self,timetable_df,merged_space_data_df):
        print("# merge - space_data + timetable_data")
        merged_timetable_df = pd.merge(timetable_df,merged_space_data_df,on=['Campus Code','Building Code','Room Code'])
        print((timetable_df.shape, merged_timetable_df.shape))
        print('Unable to merge records:',timetable_df.shape[0]-merged_timetable_df.shape[0])
        return merged_timetable_df

    def get_merged_meeting_room_usage_data(self,meeting_room_usage_df,merged_space_data_df):
        print("# merge - space_data + meeting_room_usage")
        merged_meeting_room_usage_df = pd.merge(meeting_room_usage_df,merged_space_data_df,on=['Campus Code','Building Code','Floor Code','Room Code'])
        print((meeting_room_usage_df.shape, merged_meeting_room_usage_df.shape))
        print('Unable to merge records:',meeting_room_usage_df.shape[0]-merged_meeting_room_usage_df.shape[0])
        return merged_meeting_room_usage_df

In [6]:
class DataProcessor():
    def __init__(self,uom_space_url,rm_category_type_url,
                 em_location_url,av_equipment_url,
                 timetable_2020_url,floor_name_url,
                 meeting_room_usage_url):
        self.uom_space_url = uom_space_url
        self.rm_category_type_url = rm_category_type_url
        self.em_location_url = em_location_url
        self.av_equipment_url = av_equipment_url
        self.timetable_2020_url = timetable_2020_url
        self.floor_name_url = floor_name_url
        self.meeting_room_usage_url = meeting_room_usage_url

    def load_data(self):
        self.uom_space_df = pd.read_excel(uom_space_url)
        self.rm_category_type_df = pd.read_excel(rm_category_type_url)
        self.em_location_df = pd.read_excel(em_location_url)
        self.av_equipment_df = pd.read_excel(av_equipment_url)
        self.timetable_df = pd.read_excel(timetable_2020_url, delim_whitespace=True)
        self.floor_df = pd.read_excel(floor_name_url)
        self.meeting_room_usage_df = pd.read_excel(meeting_room_usage_url)
        print('Data loaded successfully!')
        # data shapes
        print("UOM space shape:"+str(self.uom_space_df.shape))
        print("RM category:"+str(self.rm_category_type_df.shape))
        print("EM location:"+str(self.em_location_df.shape))
        print("AV equipment:"+str(self.av_equipment_df.shape))
        print("2020 timetable:"+str(self.timetable_df.shape))
        print("Floor data shape:"+str(self.floor_df.shape))
        print("Meeting room usage shape:"+str(self.meeting_room_usage_df.shape))

    def get_all_datasets(self):

        print("# clean data")
        _cleaner = DataCleaner()
        self.uom_space_df = _cleaner.clean_uom_space(self.uom_space_df)
        self.rm_category_type_df = _cleaner.clean_rm_category_type(self.rm_category_type_df)
        self.floor_df = _cleaner.clean_floor_data(self.floor_df)
        self.em_location_df = _cleaner.clean_em_location(self.em_location_df)
        self.av_equipment_df = _cleaner.clean_av_equipment(self.av_equipment_df)
        self.timetable_df = _cleaner.clean_timetable_data(self.timetable_df)
        self.meeting_room_usage_df = _cleaner.clean_meeting_room_usage(self.meeting_room_usage_df)

        print("# mutate data")
        _mutator = DataMutator()
        self.em_location_df = _mutator.mutate_em_location(self.em_location_df)
        self.timetable_df = _mutator.mutate_timetable_data(self.timetable_df)

        print("# merge data")
        _merger = DataMerger()
        self.merged_space_data = _merger.get_merged_space_data(self.uom_space_df, self.rm_category_type_df, self.floor_df)
        self.merged_em_location_data = _merger.get_merged_em_location_data(self.em_location_df,self.merged_space_data)
        self.merged_av_equipment_data = _merger.get_merged_av_equipment_data(self.av_equipment_df,self.merged_space_data)
        self.merged_timetable_data = _merger.get_merged_timetable_data(self.timetable_df,self.merged_space_data)
        self.merged_meeting_room_usage_data = _merger.get_merged_meeting_room_usage_data(self.meeting_room_usage_df,self.merged_space_data)

        return self.merged_space_data, self.merged_em_location_data, self.merged_av_equipment_data, self.merged_timetable_data, self.merged_meeting_room_usage_data

In [7]:
# local urls
uom_space_url = '../project-data/uom-space.xlsx'
rm_category_type_url = '../project-data/rm-category-type-cleaned.xlsx'
em_location_url = '../project-data/em-location.xlsx'
av_equipment_url = '../project-data/av-equipment.xlsx'
timetable_2020_url = '../project-data/2020-timetable-v2.xlsx'
floor_name_url = '../project-data/fl-name-cleaned.xlsx'
meeting_room_usage_url = '../project-data/meeting-room-usage.xlsx'

# drive urls
base_drive_url = '/content/drive/My Drive/Mission Data Science [2019-2020]/Semester 4/MAST90107/project-space-optimisation-group-3/'
uom_space_url = base_drive_url+'project-data/uom-space.xlsx'
rm_category_type_url = base_drive_url+'project-data/rm-category-type-cleaned.xlsx'
em_location_url = base_drive_url+'project-data/em-location.xlsx'
av_equipment_url = base_drive_url+'project-data/av-equipment.xlsx'
timetable_2020_url = base_drive_url+'project-data/2020-timetable-v2.xlsx'
floor_name_url = base_drive_url+'project-data/fl-name-cleaned.xlsx'
meeting_room_usage_url = base_drive_url+'project-data/meeting-room-usage.xlsx'

In [8]:
# load data
_processor = DataProcessor(uom_space_url,rm_category_type_url,em_location_url,av_equipment_url,
                           timetable_2020_url,floor_name_url,meeting_room_usage_url)
_processor.load_data()

Data loaded successfully!
UOM space shape:(22166, 14)
RM category:(209, 5)
EM location:(7709, 4)
AV equipment:(1964, 11)
2020 timetable:(131857, 23)
Floor data shape:(1369, 3)
Meeting room usage shape:(1462, 22)


In [9]:
# get datasets - after cleaning, mutation and merging
space_data, employee_data, av_equipment_data, timetable_data, mr_usage_data = _processor.get_all_datasets()

# clean data
# mutate data
# merge data
# Merge - uom_space + floor_data
((22166, 14), (22166, 15))
Unable to merge records: 0
# Merge - enhanced_uom_space + rm_category_type
((22166, 15), (22166, 18))
Unable to merge records: 0
# merge - space_data + em_location
((7709, 4), (6962, 19))
Unable to merge records: 747
# merge - space_data + av_equipment
((1964, 11), (1646, 25))
Unable to merge records: 318
# merge - space_data + timetable_data
((119419, 27), (116981, 42))
Unable to merge records: 2438
# merge - space_data + meeting_room_usage
((890, 22), (728, 36))
Unable to merge records: 162


In [13]:
class DataExtractor():

    def get_meeting_rooms_data(self,rm_category_type_df,space_data):
        # possible meeting rooms
        possible_rooms = rm_category_type_df[rm_category_type_df['Room Type'].str.contains("601|629")]
        meeting_room_types = possible_rooms['Room Type'].tolist()

        # supply of meeting rooms
        possible_meeting_rooms_df = space_data[space_data['Room Type'].isin(meeting_room_types)]
        return possible_meeting_rooms_df

    def get_toilets_data(self,rm_category_type_df,space_data):
        possible_rooms = rm_category_type_df[rm_category_type_df['Room Type Definition'].str.contains("toilet|washroom")]
        toilet_room_types = possible_rooms['Room Type'].tolist()

        possible_toilets_df = space_data[space_data['Room Type'].isin(toilet_room_types)]
        return possible_toilets_df

    def get_meeting_rooms_equipment_data(self, av_equipment_data):
        return av_equipment_data[av_equipment_data['Room Type_x'].str.contains('601|629')]

In [14]:
_extractor = DataExtractor()
possible_meeting_rooms_data = _extractor.get_meeting_rooms_data(_processor.rm_category_type_df, space_data)
possible_toilets_data = _extractor.get_toilets_data(_processor.rm_category_type_df, space_data)

In [15]:
possible_mr_data_equipments = _extractor.get_meeting_rooms_equipment_data(av_equipment_data)
possible_mr_data_equipments.shape

(125, 25)

In [16]:
possible_mr_data_equipments.groupby(by=['Campus Code','Building Code','Building Name'], as_index=False).agg({'Equipment Code': pd.Series.nunique})

,Campus Code,Building Code,Building Name,Equipment Code
0,PAR,104,ALAN GILBERT BUILDING,2
1,PAR,106,LAW BUILDING,1
2,PAR,110,THE SPOT,9
3,PAR,123,BIOSCIENCES 1,1
4,PAR,134,ELISABETH MURDOCH BUILDING,1
5,PAR,139,BABEL BUILDING,2
6,PAR,144,KENNETH MYER BUILDING,21
7,PAR,147,BIOSCIENCES 4,2
8,PAR,163,WALTER BOAS BUILDING,1
9,PAR,167,CHEMICAL ENGINEERING 2,1


## Analysing data

In [ ]:
space_data.groupby(by=['Campus Code'], as_index=False).count()

,Campus Code,Building Code,Building Name,Floor Code,Room Code,Room Name,Room Category,Room Type,Room Type Abbreviation_x,Room Capacity,Room Area m²,Room Condition 2018,Department Code,Room Location Code,Floor Name,Room Type Abbreviation_y,Description,Room Type Definition
0,BUR,246,246,246,246,14,246,246,246,198,246,198,246,246,246,246,246,246
1,CRE,369,369,369,369,14,369,369,369,268,369,359,369,369,369,369,369,369
2,DOO,776,776,776,776,19,776,776,776,374,776,599,776,776,776,776,776,776
3,PAR,17857,17857,17857,17857,1336,17857,17857,17857,17121,17857,15443,17857,17857,17857,17857,17857,17857
4,SPT,347,347,347,347,0,347,347,347,233,347,331,347,347,347,347,347,347
5,STH,1647,1647,1647,1647,356,1647,1647,1647,1538,1647,1079,1647,1647,1647,1647,1647,1647
6,WER,924,924,924,924,52,924,924,924,680,924,534,924,924,924,924,924,924


In [ ]:
possible_meeting_rooms_data.head()

,Campus Code,Building Code,Building Name,Floor Code,Room Code,Room Name,Room Category,Room Type,Room Type Abbreviation_x,Room Capacity,Room Area m²,Room Condition 2018,Department Code,Room Location Code,Floor Name,Room Type Abbreviation_y,Description,Room Type Definition
8488,PAR,102,DAVID PENINGTON BUILDING,0,G17,NaN,6,601,Conf / Meeting,20.0,53.06,Excellent,7890,PAR;102;0;G17,Ground,conf / meeting,meeting/board/conference room,a room specifically set aside for holding meet...
8489,PAR,102,DAVID PENINGTON BUILDING,2,211,NaN,6,601,Conf / Meeting,12.0,34.78,Good,7890,PAR;102;2;211,Level 2,conf / meeting,meeting/board/conference room,a room specifically set aside for holding meet...
8490,PAR,102,DAVID PENINGTON BUILDING,3,311,NaN,6,601,Conf / Meeting,12.0,34.51,Good,7890,PAR;102;3;311,Level 3,conf / meeting,meeting/board/conference room,a room specifically set aside for holding meet...
8491,PAR,102,DAVID PENINGTON BUILDING,3,366,NaN,6,601,Conf / Meeting,14.0,28.28,Excellent,7890,PAR;102;3;366,Level 3,conf / meeting,meeting/board/conference room,a room specifically set aside for holding meet...
8492,PAR,102,DAVID PENINGTON BUILDING,4,412,NaN,6,601,Conf / Meeting,16.0,34.29,Excellent,7890,PAR;102;4;412,Level 4,conf / meeting,meeting/board/conference room,a room specifically set aside for holding meet...


In [ ]:
gdf1 = possible_meeting_rooms_data.groupby(by=['Campus Code','Building Code','Building Name'], as_index=False).agg({'Room Code':pd.Series.nunique,'Room Capacity':sum})
gdf1 = gdf1.rename(columns={"Room Code": "MR_COUNT", "Room Capacity": "MR_CAP"})
gdf1['LOC_CODE']=gdf1['Campus Code']+';'+gdf1['Building Code']
gdf1

,Campus Code,Building Code,Building Name,MR_COUNT,MR_CAP,LOC_CODE
0,CRE,715,CRESWICK TEACHING LABORATORY,1,4.0,CRE;715
1,CRE,716,CRESWICK RESEARCH LABORATORIES,1,20.0,CRE;716
2,CRE,719,CRESWICK LIBRARY BUILDING,1,24.0,CRE;719
3,DOO,505,DOOKIE ACADEMIC CENTRE (MAIN BUILDING),3,26.0,DOO;505
4,DOO,527,DOOKIE - HUGH PYE BUILDING,1,10.0,DOO;527
...,...,...,...,...,...,...
111,STH,880,THE IAN POTTER SOUTHBANK CENTRE,1,14.0,STH;880
112,WER,411,WERRIBEE VETERINARY HOSPITAL,4,26.0,WER;411
113,WER,416,WERRIBEE PATHOLOGY BUILDING,1,3.0,WER;416
114,WER,417,WERRIBEE PARASITOLOGY BUILDING,1,6.0,WER;417


In [ ]:
len(gdf1[gdf1['LOC_CODE']=='PAR;142'])

1

In [ ]:
gdf1[gdf1['LOC_CODE']=='PAR;142'].iloc[0]['MR_CAP']

56.0

In [ ]:
emp_gdf = employee_data.groupby(by=['Campus Code','Building Code','Building Name'], as_index=False).agg({'Employee Sequential ID':pd.Series.nunique})
emp_gdf = emp_gdf.rename(columns={'Employee Sequential ID':'EMP_COUNT'})
emp_gdf['LOC_CODE']=emp_gdf['Campus Code']+';'+emp_gdf['Building Code']
emp_gdf

,Campus Code,Building Code,Building Name,EMP_COUNT,LOC_CODE
0,BUR,901,BURNLEY ACADEMIC CENTRE (MAIN BUILDING),31,BUR;901
1,BUR,902,BURNLEY NURSERY FACILITY,1,BUR;902
2,BUR,903,BURNLEY ENGINEERING FACILITY,15,BUR;903
3,BUR,911,BURNLEY CENTENARY CENTRE BUILDING,2,BUR;911
4,BUR,916,BURNLEY FOREST SCIENCE OFFICE,6,BUR;916
...,...,...,...,...,...
103,STH,880,THE IAN POTTER SOUTHBANK CENTRE,35,STH;880
104,WER,411,WERRIBEE VETERINARY HOSPITAL,34,WER;411
105,WER,416,WERRIBEE PATHOLOGY BUILDING,49,WER;416
106,WER,417,WERRIBEE PARASITOLOGY BUILDING,12,WER;417


In [ ]:
emp_gdf[emp_gdf['LOC_CODE']=='PAR;'].iloc[0]['EMP_COUNT']

IndexError: ignored